In [1]:
import pandas as pd
import calendar
import numpy as np

<h3>File selection - run either Training file or Test file</h3>

In [204]:
filetype1 = 'train.csv'
filetype2 = 'test.csv'

<h3>Training file</h3>

In [144]:
fname = 'data/'+filetype1
processed_fname = 'data/processed_data/processed_'+filetype1
print(fname, processed_fname)

data/train.csv data/processed_data/processed_train.csv


<h3>Test file</h3>

In [205]:
fname = 'data/'+filetype2
processed_fname = 'data/processed_data/processed_'+filetype2
print(fname, processed_fname)

data/test.csv data/processed_data/processed_test.csv


<h3>Data Processing</h3>

In [206]:
df = pd.read_csv(fname)
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,7.250339e+10,1,SHOES,3002.0
1,1,Friday,1.707711e+09,1,DAIRY,1526.0
2,1,Friday,8.947000e+10,1,DAIRY,1431.0
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0


In [207]:
df.count()

VisitNumber              653646
Weekday                  653646
Upc                      649660
ScanCount                653646
DepartmentDescription    652318
FinelineNumber           649660
dtype: int64

<h3>Drop NA values</h3>

In [208]:
df = df.dropna()
df.count()

VisitNumber              649660
Weekday                  649660
Upc                      649660
ScanCount                649660
DepartmentDescription    649660
FinelineNumber           649660
dtype: int64

In [209]:
print(df['DepartmentDescription'].nunique())
#print(df['TripType'].nunique())
print(df['FinelineNumber'].groupby(df['DepartmentDescription']).nunique())

67
DepartmentDescription
1-HR PHOTO                     13
ACCESSORIES                    88
AUTOMOTIVE                    256
BAKERY                         93
BATH AND SHOWER                81
BEAUTY                        177
BEDDING                       186
BOOKS AND MAGAZINES            73
BOYS WEAR                     179
BRAS & SHAPEWEAR               59
CAMERAS AND SUPPLIES           29
CANDY, TOBACCO, COOKIES        70
CELEBRATION                   172
COMM BREAD                     51
CONCEPT STORES                  1
COOK AND DINE                 307
DAIRY                         108
DSD GROCERY                   362
ELECTRONICS                   163
FABRICS AND CRAFTS            340
FINANCIAL SERVICES             25
FROZEN FOODS                   94
FURNITURE                      32
GIRLS WEAR, 4-6X  AND 7-14    210
GROCERY DRY GOODS             403
HARDWARE                      324
HOME DECOR                    193
HOME MANAGEMENT               177
HORTICULTURE AND ACCESS

<h3>Add new feature FLCount corresponding to number of refined categories (FinelineNumbers) for each DepartmentDescription</h3>

In [210]:
df2 = pd.DataFrame(df['FinelineNumber'].groupby(df['DepartmentDescription']).nunique())
df2 = df2.rename(columns={'FinelineNumber': "FLCount"})
df2.head()

,FLCount
DepartmentDescription,
1-HR PHOTO,13
ACCESSORIES,88
AUTOMOTIVE,256
BAKERY,93
BATH AND SHOWER,81


In [211]:
df = df.join(df2, on='DepartmentDescription')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount
0,1,Friday,7.250339e+10,1,SHOES,3002.0,464
1,1,Friday,1.707711e+09,1,DAIRY,1526.0,108
2,1,Friday,8.947000e+10,1,DAIRY,1431.0,108
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0,362


<h3>Add new feature VisitFLCount corresponding to number of refined categories (FinelineNumbers) for each VisitNumber</h3>

In [212]:
df2 = pd.DataFrame(df['FinelineNumber'].groupby(df['VisitNumber']).nunique())
df2 = df2.rename(columns={'FinelineNumber': "VisitFLCount"})
df2.head()

,VisitFLCount
VisitNumber,
1,4
2,3
3,1
4,1
6,1


In [213]:
df = df.join(df2, on='VisitNumber')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount
0,1,Friday,7.250339e+10,1,SHOES,3002.0,464,4
1,1,Friday,1.707711e+09,1,DAIRY,1526.0,108,4
2,1,Friday,8.947000e+10,1,DAIRY,1431.0,108,4
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0,362,3


<h3>Scalar encode weekday names for numerical feature</h3>

In [214]:
def encode_day_names(days):
    day_to_num = dict(zip(list(calendar.day_name), range(0, 7)))
    return [day_to_num[day] for day in days]

In [215]:
df['Weekday'] = encode_day_names(df['Weekday'])
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3


<h3>Add features corresponding to number of returns and number of purchases for a particular DepartmentDescription</h3>
<h1>*Run this part only for Training file*</h1>

In [146]:
df_groups_train = df.groupby(df['DepartmentDescription']).groups
df_groups_train.keys()

dict_keys(['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY', 'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES', 'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES', 'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD', 'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY', 'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES', 'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14', 'GROCERY DRY GOODS', 'HARDWARE', 'HEALTH AND BEAUTY AIDS', 'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL', 'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES', 'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS', 'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN', 'MEDIA AND GAMING', 'MENS WEAR', 'MENSWEAR', 'OFFICE SUPPLIES', 'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS', 'PAINT AND ACCESSORIES', 'PERSONAL CARE', 'PETS AND SUPPLIES', 'PHARM

In [147]:
df_groups_train

{'1-HR PHOTO': Int64Index([ 10077,  11068,  11069,  11070,  11071,  12084,  17148,  17149,
              18033,  27328,
             ...
             626971, 627185, 629994, 632767, 632768, 633399, 633761, 635443,
             635525, 643445],
            dtype='int64', length=394),
 'ACCESSORIES': Int64Index([    60,   1778,   2264,   2882,   3531,   5273,   5348,   5349,
               5546,   6039,
             ...
             640959, 640962, 642422, 642435, 642436, 644662, 644699, 646102,
             646687, 646689],
            dtype='int64', length=1318),
 'AUTOMOTIVE': Int64Index([   144,    146,    545,    553,    555,    556,    953,    968,
                969,    970,
             ...
             646205, 646226, 646372, 646587, 646588, 646589, 646590, 646623,
             646861, 646938],
            dtype='int64', length=5414),
 'BAKERY': Int64Index([    85,     86,     90,    129,    288,    313,    314,    464,
                499,    502,
             ...
            

<h1>*End*</h1>

In [216]:
df_groups = df.groupby(df['DepartmentDescription']).groups
data = []
for dept_desc, ids in df_groups.items():
    #print(dept_desc)
    returns = 0
    purchases = 0
    for idx in ids:
        if df.loc[idx]['ScanCount'] < 0:
            returns += df.loc[idx]['ScanCount']
        else:
            purchases += df.loc[idx]['ScanCount']
    data.append({"DepartmentDescription":dept_desc, "NumReturns":returns, "NumPurchases":purchases})

In [217]:
df2 = pd.DataFrame(data)
df2['NumReturns'] = abs(df2['NumReturns'])
df2.head()

,DepartmentDescription,NumPurchases,NumReturns
0,1-HR PHOTO,498,38
1,ACCESSORIES,1421,91
2,AUTOMOTIVE,5916,383
3,BAKERY,8337,111
4,BATH AND SHOWER,5129,201


In [218]:
df = df.merge(df2, how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6114,291
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52219,409
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52219,409
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92682,854
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81132,833


<h3>Add features corresponding to number of returns and number of purchases for a particular VisitNumber</h3>

In [219]:
df_groups = df.groupby(df['VisitNumber']).groups
len(df_groups.keys())

94288

In [220]:
df_groups

{1: Int64Index([0, 1, 2, 3], dtype='int64'),
 2: Int64Index([4, 5, 6, 7], dtype='int64'),
 3: Int64Index([8, 9], dtype='int64'),
 4: Int64Index([10], dtype='int64'),
 6: Int64Index([11, 12], dtype='int64'),
 13: Int64Index([13, 14], dtype='int64'),
 14: Int64Index([15, 16, 17, 18, 19, 20, 21, 22, 23, 24], dtype='int64'),
 16: Int64Index([25, 26, 27, 28, 29, 30, 31, 32], dtype='int64'),
 18: Int64Index([33, 34, 35, 36, 37, 38, 39], dtype='int64'),
 21: Int64Index([40, 41, 42, 43, 44, 45, 46], dtype='int64'),
 22: Int64Index([47], dtype='int64'),
 24: Int64Index([48], dtype='int64'),
 27: Int64Index([49, 50, 51, 52, 53, 54, 55, 56], dtype='int64'),
 34: Int64Index([57, 58], dtype='int64'),
 35: Int64Index([59], dtype='int64'),
 36: Int64Index([60, 61], dtype='int64'),
 37: Int64Index([62, 63, 64], dtype='int64'),
 38: Int64Index([65], dtype='int64'),
 39: Int64Index([66, 67], dtype='int64'),
 44: Int64Index([68], dtype='int64'),
 46: Int64Index([69, 70], dtype='int64'),
 48: Int64Index([

In [221]:
df_groups = df.groupby(df['VisitNumber']).groups
data = []
for visit_id, ids in df_groups.items():
    #print(dept_desc)
    returns = 0
    purchases = 0
    for idx in ids:
        if df.loc[idx]['ScanCount'] < 0:
            returns += df.loc[idx]['ScanCount']
        else:
            purchases += df.loc[idx]['ScanCount']
    data.append({"VisitNumber":visit_id, "VisitNumReturns":returns, "VisitNumPurchases":purchases})

In [222]:
df2 = pd.DataFrame(data)
df2['VisitNumReturns'] = abs(df2['VisitNumReturns'])
df2.head()

,VisitNumPurchases,VisitNumReturns,VisitNumber
0,4,0,1
1,4,0,2
2,1,1,3
3,1,0,4
4,1,1,6


In [223]:
df = df.merge(df2, how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,VisitNumPurchases,VisitNumReturns
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6114,291,4,0
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52219,409,4,0
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52219,409,4,0
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92682,854,4,0
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81132,833,4,0


<h3>Min, max, mean number of products for a DepartmentDescription</h3>

In [224]:
df_groups = df.groupby(df['DepartmentDescription']).groups
data = []
for dept_desc, ids in df_groups.items():
    data.append({"DepartmentDescription":dept_desc, "MinD":min(df['FLCount']), "MaxD":max(df['FLCount']), "MeanD": np.mean(df['FLCount'])})

In [225]:
df2 = pd.DataFrame(data)
df = df.merge(df2, how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,VisitNumPurchases,VisitNumReturns,MaxD,MeanD,MinD
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6114,291,4,0,499,255.78492,1
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52219,409,4,0,499,255.78492,1
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52219,409,4,0,499,255.78492,1
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92682,854,4,0,499,255.78492,1
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81132,833,4,0,499,255.78492,1


<h3>Min, max, mean number of products for a VisitNumber</h3>

In [226]:
df_groups = df.groupby(df['DepartmentDescription']).groups
data = []
for dept_desc, ids in df_groups.items():
    data.append({"DepartmentDescription":dept_desc, "MinV":min(df['VisitFLCount']), "MaxV":max(df['VisitFLCount']), "MeanV": np.mean(df['VisitFLCount'])})

In [227]:
df2 = pd.DataFrame(data)
df = df.merge(df2, how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,VisitNumPurchases,VisitNumReturns,MaxD,MeanD,MinD,MaxV,MeanV,MinV
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6114,291,4,0,499,255.78492,1,120,15.235902,1
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52219,409,4,0,499,255.78492,1,120,15.235902,1
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52219,409,4,0,499,255.78492,1,120,15.235902,1
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92682,854,4,0,499,255.78492,1,120,15.235902,1
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81132,833,4,0,499,255.78492,1,120,15.235902,1


<h3>Count how many items belonging to each department description are bought in each trip</h3>

In [228]:
departments = df.groupby(df['DepartmentDescription']).groups
departments.keys()

dict_keys(['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY', 'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES', 'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES', 'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD', 'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY', 'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES', 'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14', 'GROCERY DRY GOODS', 'HARDWARE', 'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL', 'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES', 'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS', 'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN', 'MEDIA AND GAMING', 'MENS WEAR', 'MENSWEAR', 'OFFICE SUPPLIES', 'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS', 'PAINT AND ACCESSORIES', 'PERSONAL CARE', 'PETS AND SUPPLIES', 'PHARMACY OTC', 'PHARMACY RX', '

In [229]:
visits_by_group = df.groupby(['VisitNumber','DepartmentDescription'])['DepartmentDescription'].count()
visits_by_group.head()

VisitNumber  DepartmentDescription
1            DAIRY                    2
             GROCERY DRY GOODS        1
             SHOES                    1
2            BAKERY                   1
             DSD GROCERY              1
Name: DepartmentDescription, dtype: int64

In [230]:
#visits_by_group[(8, 'PAINT AND ACCESSORIES')]

In [231]:
data = []
for visit, dept in visits_by_group.keys():
    #print(visit, dept)
    data.append({"VisitNumber":visit, dept:visits_by_group[(visit,dept)]})
    #print(data)
len(data)

320936

In [232]:
data

[{'DAIRY': 2, 'VisitNumber': 1},
 {'GROCERY DRY GOODS': 1, 'VisitNumber': 1},
 {'SHOES': 1, 'VisitNumber': 1},
 {'BAKERY': 1, 'VisitNumber': 2},
 {'DSD GROCERY': 1, 'VisitNumber': 2},
 {'IMPULSE MERCHANDISE': 2, 'VisitNumber': 2},
 {'PERSONAL CARE': 2, 'VisitNumber': 3},
 {'FABRICS AND CRAFTS': 1, 'VisitNumber': 4},
 {'BOYS WEAR': 2, 'VisitNumber': 6},
 {'DSD GROCERY': 1, 'VisitNumber': 13},
 {'PRE PACKED DELI': 1, 'VisitNumber': 13},
 {'CELEBRATION': 1, 'VisitNumber': 14},
 {'DSD GROCERY': 3, 'VisitNumber': 14},
 {'FROZEN FOODS': 2, 'VisitNumber': 14},
 {'HOUSEHOLD CHEMICALS/SUPP': 1, 'VisitNumber': 14},
 {'IMPULSE MERCHANDISE': 1, 'VisitNumber': 14},
 {'MENS WEAR': 2, 'VisitNumber': 14},
 {'DSD GROCERY': 5, 'VisitNumber': 16},
 {'FABRICS AND CRAFTS': 1, 'VisitNumber': 16},
 {'IMPULSE MERCHANDISE': 1, 'VisitNumber': 16},
 {'PRODUCE': 1, 'VisitNumber': 16},
 {'DSD GROCERY': 3, 'VisitNumber': 18},
 {'FROZEN FOODS': 1, 'VisitNumber': 18},
 {'GROCERY DRY GOODS': 1, 'VisitNumber': 18},
 {'

In [233]:
feature_names = [d for d in df_groups_train.keys()]
feature_names.append("VisitNumber")
#df2 = pd.DataFrame(columns=feature_names).set_index("VisitNumber")
df2 = df2.from_records(data, columns=feature_names).fillna(0).set_index("VisitNumber")
df2 = df2.groupby("VisitNumber").sum()
df2.head()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [234]:
df["VisitNumber"].nunique()

94288

In [235]:
df = df.join(df2,on="VisitNumber", how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6114,291,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52219,409,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52219,409,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92682,854,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81132,833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
df_final = df.groupby("VisitNumber").mean()
df_final.head()

,Weekday,Upc,ScanCount,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,VisitNumPurchases,VisitNumReturns,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,4.0,6.304308e+10,1.0,2378.50,270.75,4.0,50808.50,490.75,4.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,4.0,4.640786e+10,1.0,6368.25,169.75,3.0,38308.75,476.50,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,7.410811e+09,0.0,4504.00,305.00,1.0,44890.00,808.00,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,7.287926e+09,1.0,5924.00,340.00,1.0,5986.00,271.00,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,7.603139e+09,0.0,654.00,179.00,1.0,3943.00,163.00,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3>Convert to CSV</h3>

In [237]:
df.to_csv(processed_fname)